# SDL.1 Exploring Data in Data Lab

### Objective: Learn how to search for tags, pull data into a Jupyter notebook, perform a calculation, and push data and formulas to Seeq.

### Scenario: You have developed an algorithm to calculate the health of a Cooling Tower based on temperature data within each area.  In the first step, identify the temperature tags, and cleanse the data for input into your algorithm. 

## Step 0: Import Appropriate Libraries and Set Compatibility

When using Data Lab, the Seeq SPy library is imported by default; it is not necessary to do so here.
Start by importing other libraries as shown below.

It's important to specify what major version of SPy your code was written for to allow it to make choices about what behavior is going to be most compatible in the future. This prevents your code from breaking as SPy continues to get updated. Run `spy.__version__` in a cell to find what version of SPy you have installed.

In [ ]:
import pandas as pd

# Set the compatibility option so that you maximize the chance that SPy will remain compatible with your notebook/script
spy.options.compatbility = 188

<div class="alert alert-block alert-info">
    <b>Tip:</b> Press <b>Shift + Enter</b> to run a cell.
</div>

***

## Step 1: Search for data from Seeq

Let's search for signals using Seeq's Example Data; finding data on your server will be similar.

<div class="alert alert-block alert-success">
<b>Instructor Note:</b> Explore the Data Tab in Seeq Workbench. Search for <tt>Area ?_Temperature</tt>, filter the results by the <tt>Example Data</tt>, and order the list alphabetically by <tt>Name</tt>. </div>

Now, **search** for the Temperature signals using `spy.search()` function. This will retrieve metadata (information) about all the signals. Click the link for more information on [spy.search](https://python-docs.seeq.com/user_guide/spy.search.html)

In [ ]:
#search for all signals named 'Area ?_Temperature' in the 'Example Data' Datasource Name
search = spy.search(query = {'Name': 'Area ?_Temperature',
                     'Datasource Name': 'Example Data'}
                    , order_by = 'Name')

# The .head() funtion prints the first 5 rows of a dataframe. 
search.head()

<div class="alert alert-block alert-info">
<b>Tip:</b> For function documentation, place the cursor in the function and hit <b>Shift + Tab</b>.
</div>

In [ ]:
# The .head() function can be used to filter down to Area A, Area B, and Area C.
search = search.head(3)

search

***

## Step 2: Pull data from Seeq into Data Lab

Now that we have searched for the tags in Cooling Tower 1, let's **pull** the data into this jupyter notebook. Click the link for more information on [spy.pull](https://python-docs.seeq.com/user_guide/spy.pull.html#)

In [ ]:
my_data = spy.pull(items = search, 
    start='2022-01-01', 
    end='2022-01-07')

my_data.head()

<div class="alert alert-block alert-success">
<b>Instructor Note:</b> Seeq's calculation engine will interpolate to produce a uniform DataFrame with a default <tt> grid='15min'</tt>. Specify the grid parameter to control the granularity of the data. Specify <tt> grid=None </tt> if you want the raw data.  (ref Function Documentation) </div>

**Optional Exercise:** Update sampling frequency to `grid='5min'` and the `start` and `end` time to the last 7 days.

Let's take a peek at the data we pulled using the built in `DataFrame.plot()` function.

 

In [ ]:
my_data.plot()

Notice the outliers in Area C.  For the algorithm to calculate, the outliers will need to be removed. The engineers have provided a calculation using Seeq's [formula language](https://support.seeq.com/space/KB/54231048/Seeq+Formula+Language) to cleanse the temperature signals. 

<div class="alert alert-block alert-warning">
<b>Discussion Topic:</b> When should you make visualizations in Data Lab vs Workbench? 
    <details>
    <summary>✼</summary>
<i>Due to ease of use and because calculations update automatically as new data comes in, it is usually preferable to make visualizations in Workbench when possible (including trends, deviations from limits, aggregations and metrics, scatterplots, scorecard tables, bar graphs and histograms, predictions, and more).</i>
</details>
</div>



<div class="alert alert-block alert-warning">
<b>Discussion Topic:</b> What is the difference between <tt>spy.pull()</tt> and <tt>spy.search()</tt>?
    <details>
  <summary>✼</summary>

`spy.search()` <i> results in the name, description, unit of measure, datasource name, etc.
    
`spy.pull()` <i> results in the time stamps and corresponding signal values
        
#### Data vs Metadata

There are two main types of information processed by Seeq: _Data_ and _metadata_:

- **Data** is the time series and time interval information that is either collected or derived from sensor data. It consists of timestamps and values (samples), or time intervals and properties (capsules). This data can be plotted on a trend or used to train a neural network, for example.

- **Metadata** is the information about the data, that is independent of a particular point in time or time interval. For example, a signal's _name_, _description_ and _unit of measure_ is classified as metadata; or the formula that is used to derive a new signal from one or more source signals; or the asset tree that is used to model similar equipment or industrial processes.
    </details>
</details>
</div>



***

## Step 3: Pull data from Seeq and perform calculations in Data Lab

Using `spy.pull()` you can apply calculations from Seeq's formula language at the same time as pulling the data. Apply the cleansing formula provided by the engineering team: `'$signal.remove(($signal < 0).merge(15min)).agilefilter(5min).tolinear(15min)'`

In [ ]:
my_calculated_data = spy.pull(items = search, 
    start='2022-01-01', 
    end='2022-01-07', 
    calculation='$signal.remove(($signal < 0).merge(15min)).agilefilter(5min).tolinear(15min)',
    grid='5min', 
    header='Name')

# Rename the calculated signals with an approrpiate description
my_calculated_data = my_calculated_data.add_suffix('_Cleansed')

my_calculated_data.head()

<div class="alert alert-block alert-success">
<b>Instructor Note:</b> When applying a calculation to a signal/condition/scalar, calculation must be a string with a single variable in it (ref Function Documentation).
</div>

In [ ]:
my_calculated_data.plot()

***
## Step 4: Push Calculated Data to Seeq
Now, we can push the calculated data back to Seeq. Let's go to the workbench and view the data.

Click the link for more information on [spy.push](https://python-docs.seeq.com/user_guide/spy.push.html#)

In [ ]:
spy.push(data = my_calculated_data)

<div class="alert alert-block alert-warning">
<b>Discussion Topic:</b> What happens if you expand the display range in your new Workbench to 14 days?
</div>

<div class="alert alert-block alert-warning">
<b>Discussion Topic:</b> Where in Seeq did the data get pushed to? Why?
    <details>    
     <summary>✼</summary>
<i>The default parameters for the workbook, worksheet, and datasource are:</i> <tt>workbook = 'Data Lab >> Data Lab Analysis', worksheet = 'From Data Lab', datasource = 'Seeq Data Lab'</tt><i>(see function documentation).
    </details>
</div>

<div class="alert alert-block alert-info">
<b>Tip:</b> Saving the push_result data to a varaible allows you to troubleshoot the data within the table. For example, if pushes were not successful, we can filter on the Push Result column to find the problem.    
</div>

***

## Step 5: Push Single Formula to Seeq
In the previous step, we pushed data for a specified period. In this step, we will push the formula (or metadata) back to Seeq. Let’s push back the cleansed signal for Area C, to remove the outliers.


Everything in Seeq has a unique ID attached to it.  If we look at the search dataframe we created, we can see the ID in the first column. We can manually use the ID for further calculations or find the ID programmatically using dataframes.

In [ ]:
search

<div class="alert alert-block alert-success">
<b>Instructor Note:</b> Go to workbench and investige the Item Properties for <tt>Area C_Temperature</tt>. Notice the item ID is the same as in the table above.
</div>

In [ ]:
# Filter the search data frame
area_c_id = search[search['Name'] == 'Area C_Temperature']
area_c_id

In [ ]:
# Push the formula by creating a datadrame
spy.push(metadata = pd.DataFrame([{
    'Name': 'Area C_Cleansed_formula',
    'Formula': '$signal.remove(($signal < 0).merge(15min)).agilefilter(5min).tolinear(15min)',
    'Formula Parameters': {'$signal':area_c_id}
}]))

In this exercise we are pushing a single formula back to Seeq. In Exercise SDL.3, we will show you a simple way how to push formulas on multiple signals. 

<div class="alert alert-block alert-warning">
<b>Discussion Topic:</b> What happened to the cleansed data from SDL.1 - Step 4: Push Calculated Data to Seeq?
    <details>    
     <summary>✼</summary>
<i>The default parameters for the workbook, worksheet, and datasource are:</i> <tt>workbook = 'Data Lab >> Data Lab Analysis', worksheet = 'From Data Lab', datasource = 'Seeq Data Lab'</tt><i>(see function documentation).  The worksheet will only display the content from the last</i> <tt>`spy.push()</tt><i>.  The content of previous pushes can still be found by searching for the signal names and/or the datasource. </i>
    </details>
</div>


**Optional Exercise:** Search for Area A_Temperature_Cleansed in your workbench.

**Optional Exercise:** Include the worksheet parameter in the spy.push code and re-push: `worksheet = 'Push Single Formula'`. 

<div class="alert alert-block alert-warning">
<b>Discussion Topic:</b> When would you push back data vs a formula? What is the best approach in this scenario?
</div>

***

## Step 6: Push Multiple Formulas to Seeq (Optional)

In the last step of this exercise, we use a data frame to push back multiple formulas, instead of only one formula at a time. This requires knowledge of python dataframes, which is not taught in this course. 



In [ ]:
# Creates a copy of the search table so we can manipulate it
formulas = search.copy()

formulas

Using dataframes, we will rename the signals, supply the formula and provide the item ID. Notice the syntax required for the formula parameters `$signal = 'ID'`

In [ ]:
formulas['Name'] = formulas['Name'] + '_Cleansed_dataframe'
formulas['Formula'] = '$signal.remove(($signal < 0).merge(15min)).agilefilter(5min).tolinear(15min)'
formulas['Formula Parameters'] = '$signal=' + formulas['ID']

<div class="alert alert-block alert-success">
<b>Instructor Note:</b> Compare the original search table and the new formula table. See the addition of the Formula and Formula Parameters column, and the updated Name column.</div>

In Jupyter Notebooks `$` are used to format text to *italics*. If you look at the formula in the table above, the `$` are missing and the text is formatted to italics.



In [ ]:
spy.push(metadata = formulas[['Name', 'Formula', 'Formula Parameters']],
         worksheet='Push Multiple Formulas')

<div class="alert alert-block alert-info">
    <b>Tip:</b> Refresh ↻ your workbench screen to see results from a push to a new worksheet (or click link directly from Juypter notebook). 
</div>

***
## SDL.1 Summary

The steps above have been combined into two summary cells. One block for pushing data to Seeq and the second for pushing formulas back to seeq. This summary code performs the same actions, with slight modifications for Cooling Tower 2. 


### Push Data to Seeq

In [ ]:
# Step 1: Search for data from Seeq
search = spy.search(query = {'Datasource Name': 'Example Data',
                     'Name': "Area ?_Temperature"})
search2 = search[search['Name'].str.contains('D|E')]

# Step 3: Pull data from Seeq and perform calculations in Data Lab
my_calculated_data2 = spy.pull(
    items = search2, 
    start='2022-01-01', 
    end='2022-01-07', 
    calculation='$signal.remove(($signal < 0).merge(15min)).agilefilter(5min).tolinear(15min)',
    grid='5min', 
    header='Name')
my_calculated_data2 = my_calculated_data2.add_suffix('_Cleansed')

# Step 4: Push Calculated Data to Seeq
spy.push(data = my_calculated_data2, worksheet='Push Calculated Data CT2')

### Push Metadata to Seeq

In [ ]:
# Step 1: Search for data from Seeq
search = spy.search(query = {'Datasource Name': 'Example Data',
                     'Name': "Area ?_Temperature"})
search2 = search[search['Name'].str.contains('D|E')]

# Step 6: Create Data Frame
formulas = search2.copy()
formulas['Formula'] ='$signal.remove(($signal < 0).merge(15min)).agilefilter(5min).tolinear(15min)'
formulas['Name'] = formulas['Name'] + '_Cleansed_dataframe'
formulas['Formula Parameters'] = '$signal=' + formulas['ID']

# Step 6: Push Multiple Formulas to Seeq
spy.push(metadata = formulas[['Name', 'Formula', 'Formula Parameters']],
         worksheet='Push Multiple Formulas CT2')